In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, Nadam, Adagrad, SGD, RMSprop
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv("sensor-nodes.csv")
df.head()


In [ ]:
X = df.drop(columns=['Node_ID', 'Timestamp', 'IP_Address', 'Is_Malicious'])
y = df['Is_Malicious']


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

print("Shape of X_train:", X_train.shape)


In [ ]:
sequence_length = X_train.shape[1]
num_features = 1

X_train_reshaped = X_train.reshape(-1, sequence_length, num_features)
X_val_reshaped = X_val.reshape(-1, sequence_length, num_features)
X_test_reshaped = X_test.reshape(-1, sequence_length, num_features)


In [ ]:
model = Sequential()

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(
    sequence_length, num_features), kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))

model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))

# model.add(Dense(1, activation='sigmoid'))


In [ ]:
adam_optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=adam_optimizer,
              loss='binary_crossentropy', metrics=['accuracy'])

adam_history = model.fit(X_train_reshaped, y_train, epochs=30,
                         batch_size=32, validation_data=(X_val_reshaped, y_val))


In [ ]:
nadam_optimizer = Nadam(learning_rate=0.001)

model.compile(optimizer=nadam_optimizer,
              loss='binary_crossentropy', metrics=['accuracy'])

nadam_history = model.fit(X_train_reshaped, y_train, epochs=30,
                          batch_size=32, validation_data=(X_val_reshaped, y_val))


In [ ]:
adagrad_optimizer = Adagrad(learning_rate=0.001)

model.compile(optimizer=adagrad_optimizer,
              loss='binary_crossentropy', metrics=['accuracy'])

adagrad_history = model.fit(X_train_reshaped, y_train, epochs=30,
                            batch_size=32, validation_data=(X_val_reshaped, y_val))


In [ ]:
sgd_optimizer = SGD(learning_rate=0.001)

model.compile(optimizer=sgd_optimizer,
              loss='binary_crossentropy', metrics=['accuracy'])

sgd_history = model.fit(X_train_reshaped, y_train, epochs=30,
                        batch_size=32, validation_data=(X_val_reshaped, y_val))


In [ ]:
rms_optimizer = RMSprop(learning_rate=0.001)

model.compile(optimizer=rms_optimizer,
              loss='binary_crossentropy', metrics=['accuracy'])

rms_history = model.fit(X_train_reshaped, y_train, epochs=30,
                        batch_size=32, validation_data=(X_val_reshaped, y_val))


In [ ]:
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(adam_history.history['loss'], label='Training Loss')
plt.plot(adam_history.history['val_loss'], label='Validation Loss')

plt.title("Training and Validation Loss")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(adam_history.history['accuracy'], label='Training Accuracy')
plt.plot(adam_history.history['val_accuracy'], label='Validation Accuracy')

plt.title("Training and Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(nadam_history.history['loss'], label='Training Loss')
plt.plot(nadam_history.history['val_loss'], label='Validation Loss')

plt.title("Training and Validation Loss")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(nadam_history.history['accuracy'], label='Training Accuracy')
plt.plot(nadam_history.history['val_accuracy'], label='Validation Accuracy')

plt.title("Training and Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(adagrad_history.history['loss'], label='Training Loss')
plt.plot(adagrad_history.history['val_loss'], label='Validation Loss')
plt.title("Training and Validation Loss")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(adagrad_history.history['accuracy'], label='Training Accuracy')
plt.plot(adagrad_history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Training and Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(sgd_history.history['loss'], label='Training Loss')
plt.plot(sgd_history.history['val_loss'], label='Training Loss')

plt.title("Training and Validation Loss")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(sgd_history.history['accuracy'], label='Training Accuracy')
plt.plot(sgd_history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Training and Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(rms_history.history['loss'], label='Training Loss')
plt.plot(rms_history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(rms_history.history['accuracy'], label='Training Accuracy')
plt.plot(rms_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.plot(adam_history.history['loss'],
         label="Adam Training Loss", color="blue")


plt.plot(adam_history.history['val_loss'],
         label="Adam Validation Loss", color="lightblue")


plt.plot(nadam_history.history['loss'],
         label="Nadam Training Loss", color="red")
plt.plot(nadam_history.history['val_loss'],
         label="Nadam Validation Loss", color="lightcoral")

plt.plot(adagrad_history.history['loss'],
         label="Adagrad Training Loss", color="green")
plt.plot(adagrad_history.history['val_loss'],
         label="Adagrad Validation Loss", color="lightgreen")

plt.plot(sgd_history.history['loss'],
         label="SGD Training Loss", color="yellow")
plt.plot(sgd_history.history['val_loss'],
         label="SGD Validation Loss", color="gold")

plt.plot(rms_history.history['loss'],
         label="RMS Training Loss", color="pink")
plt.plot(rms_history.history['val_loss'],
         label="RMS Validation Loss", color="purple")

plt.title("Training and Validation loss comparison for different optimizers")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.plot(adam_history.history['accuracy'],
         label="Adam Training Accuracy", color="blue")


plt.plot(adam_history.history['val_accuracy'],
         label="Adam Validation Accuracy", color="lightblue")


plt.plot(nadam_history.history['accuracy'],
         label="Nadam Training Accuracy", color="red")


plt.plot(nadam_history.history['val_accuracy'],
         label="Nadam Validation Accuracy", color="lightcoral")


plt.plot(adagrad_history.history['accuracy'],
         label="Adagrad Training Accuracy", color="green")


plt.plot(adagrad_history.history['val_accuracy'],
         label="Adagrad Validation Accuracy", color="lightgreen")

plt.plot(sgd_history.history['accuracy'],
         label="SGD Training Accuracy", color="yellow")
plt.plot(sgd_history.history['val_accuracy'],
         label="SGD Validation Accuracy", color="gold")\


plt.plot(rms_history.history['accuracy'],
         label="RMS Training Accuracy", color="pink")
plt.plot(rms_history.history['val_accuracy'],
         label="RMS Validation Accuracy", color="purple")

plt.title("Training and Validation accuracy comparison for different optimizers")


plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()


plt.grid(True)


plt.show()
